[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/jun-ogura/preprocessing-pet-images/blob/master/animal_PET_img_processing_ver01.ipynb)

# Pythonを使ったPETデータ前処理
## - Main functions -
---

　※　一般的にPET画像処理は有償ソフトウェアのPMODを使用することが多いですが、<br>
　　　ここでは**全ての処理を無料のツールを使って行う方法**についてまとめて<br>
　　　います。<br>
<br>
　※　ここでは私が普段使用している動物PET/CT画像（ClairvivoPET/CT, 島津製作所）<br>
　　　の処理を前提として記載していますが、他の機種やモダリティ画像およびヒトの<br>
　　　PET/CT/MRI画像等にも利用できるものがあります（自身の使用している機種・<br>
　　　画像に合わせてパラメーターを変更してください）。<br>
<br>
　※　ClairvivoPET/CT（島津製作所）は一体型ではなくPETとCTで別々の装置になります。<br>
<br>
　※　一部の処理はdynamic imageに未対応です（今後対応予定）。

# 01. パッケージの読み込み
---
各関数で必要となるパッケージは以下です。<br>
- numpy
- nibabel
- nilearn
- pandas

以下のコードを実行してパッケージを読み込みます。

In [ ]:
import os
import numpy as np
import nibabel as nib
from nibabel.processing import smooth_image
from nilearn.image import load_img, mean_img, index_img, resample_img, concat_imgs, iter_img
from nilearn.regions import img_to_signals_labels
import pandas as pd

# 02. 回転・平行移動をするための関数
---
　ClairvivoPET/CTのデータはLR軸を中心に‐90度回転させることで、一般公開されている動物用の各種テンプレート画像と軸が合うようになります（SPMで画像を表示したとき、一般的なテンプレート画像と軸が合うように修正します）。<br>
<br>
　また、一般的なテンプレート画像と座標が大きくずれているため手動での修正がし辛いことが多々あります。<br>
位置合わせの修正にかかる負担を最小限とするために、取得した画像の座標をおおまかに修正します。<br>
<br>
　大まかに修正された画像のより正確な位置合わせは[ 3D Slicer ](https://www.slicer.org/)を使ってください。

In [ ]:
def AffineTransform(inputDir, inputFile, TransformMatrix, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #TransformMatrix : affine変換行列（4x4）
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()

    # affine変換後の回転成分を計算
    # 変換行列にオリジナルのaffine行列をかける、順番に注意
    newAffineMatrix = np.dot(TransformMatrix, niimg.header.get_best_affine())

    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(ImgArryData, affine=newAffineMatrix, header=new_header)
    print('Completed: ' + '/' + inputFile + '.nii')

    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

上記の関数を使用するために入力引数 TransformMatrix の設定をします。<br>
<br>
以下の数値は[ SAMIT](https://mic-umcg.github.io/samit/)（SPM用のtoolbox）に含まれる " Schwarz_T2w.nii " （SDラット）をテンプレートとした場合のものになります。<br>
<br>
他のテンプレート画像を使用する場合は適宜変更します。<br>
<br>
最終的には3D Slicerで調整を行います。<br>
<br>
なお、入力引数 TransformMatrix は3次元座標における回転行列と平行移動行列を表したものになります。<br>
<br>


In [ ]:
# Clairvivo PET用
LR_90rotate = np.array([[1, 0, 0, -3],
                        [0, 0, 1, 180],
                        [0, -1, 0, -15],
                        [0, 0, 0, 1]])

# Clairvivo CT用
LR_90rotate_CT = np.array([[1, 0, 0, 0],
                           [0, 0, 1, 180],
                           [0, -1, 0, -14],
                           [0, 0, 0, 1]])

以下に具体的な使用例を示します。<br>
このJupyter Notebookファイルと同じフォルダ階層に以下のフォルダが存在し、<br>
そのフォルダ内にNIfTIファイルが保存されているとします。

    ./static/test_data.nii

最初に関数に渡す引数（inputDir、outputDir、prefix、suffix）を指定します。<br>
関数を実行すると処理結果の画像ファイルが引数で指定されたフォルダに出力されます。<br>
<br>
各引数の説明は上記の関数内に記載されていますのでそちらをご確認ください（以降の関数も同様）。

In [ ]:
inputDir = './static'
outputDir = './static'
prefix = 'r_'
suffix = ''

AffineTransform(inputDir, 'test_data', LR_90rotate, outputDir, prefix, suffix)

# 03-1. create SUV image
---
　上記02で作製した画像を体重（g）とトレーサーの最終投与量（MBq）を使ってSUV画像を作製します。

　SUVの計算式は以下の通りです。

$$
SUV=\frac{uptake\;in\;the\;Image}{\frac{Injected\;dose}{body\;weight}}
$$

In [ ]:
def createSUVimg(inputDir, FileName, Weight_g, ID_MBq, outputDir, prefix, suffix):
    # inputDir: 
    # FileName: 拡張子は含めない
    # Weight_g: 
    # ID_MBq: Final Injection Dose (MBq) of tracer
    # outputDir: 出力ディレクトリパス
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語
    
    # load a non-SUV NIfTI image
    tmpPathName = inputDir + '/' + FileName + '.nii'
    niimg = nib.load(tmpPathName)

    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryData = niimg.get_fdata()

    # 体重と最終投与量をもとにSUVを計算する
    # MBq -> kBq
    # g -> kg
    CalcImg = LoadImgArryData/((ID_MBq*1000)/(Weight_g/1000))
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, None, header=new_header)
    print('Completed: ' + '/' + FileName + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + FileName + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下は実行例

In [ ]:
in_outDir = './static'
prefix = ''
suffix = '-SUV'

createSUVimg(in_outDir, 'r_test_data', 230.0,  25.0, in_outDir, prefix, suffix)

# 03-2. create %ID/ml image
---
　上記02で作製した画像からトレーサーの最終投与量（MBq）を使って%ID/ml画像を作製します。

　%ID/mlの計算式は以下の通りです。

$$
\%ID/ml=\frac{uptake\;in\;the\;Image}{Injected\;dose}
$$


In [ ]:
def create_PrecentID_per_ml_img(inputDir, FileName, ID_MBq, outputDir, prefix, suffix):
    # inputDir: 
    # FileName: 拡張子は含めない
    # ID_MBq: Final Injection Dose (MBq) of tracer
    # outputDir: 出力ディレクトリパス
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語
    
    # load a non-SUV NIfTI image (unit: Bq/ml)
    tmpPathName = inputDir + '/' + FileName + '.nii'
    niimg = nib.load(tmpPathName)

    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryData = niimg.get_fdata()

    # 最終投与量をもとに %ID/ml を計算する
    # もとの画素値がBq単位なので、最終投与濃度をMBq > Bqに変換して計算
    CalcImg = (LoadImgArryData/(ID_MBq*10**6))*100
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, None, header=new_header)
    print('Completed: ' + '/' + FileName + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + FileName + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下は実行例です。

In [ ]:
in_outDir = './static'
prefix = ''
suffix = '-IDml'

create_PrecentID_per_ml_img(in_outDir, 'r_test_data',  25.0, in_outDir, prefix, suffix)

# 04. 3D Slicerを用いてテンプレート画像に位置合わせを行う
---
　上記03で作製した画像の位置合わせを行う。<br>


　すでに02の工程で大まかな位置合わせができているため、3D Slicerで細かい修正を行う。<br>



# 05. SPM12（standalone版）を用いてNormalise処理を行う
---
　脳画像について集団解析を行う場合には空間標準化が必要である。<br>
　各個体のMRIの構造画像を持っているのであればSPM12（standalone版）などを用いて空間標準化を行う。<br>
<br>
　[SPM12（standalone版）](https://www.fil.ion.ucl.ac.uk/spm/software/download/)は無料のMatlabランタイム上で動くため、Matlabは必要ない。
　ダウンロードフォームでSPM Version＞SPM Standalone を選択するとダウンロードできる。<br>
<br>
　SPMでnormalise処理を行うと出力される画像は自動的にCroppingされている。<br>
　そのため以下の06.Cropping処理は行う必要はない。

# 06. Cropping処理
---
　VOI解析等を行う際に、全ての画像で同じようにCroppingされていないと正しい結果が得られないことがある（VOIテンプレート画像と処理したい画像をフュージョンさせ、位置を合わせてもそれぞれの画像の大きさが異なる場合は正しい結果が得られないことがある）。<br>
<br>
　SPMなどのように自動でCropping処理されるソフトウェアを使っていない場合は、適宜この処理を実行する必要がある。<br>
<br>
　下の参照リンクをもとに作成・改良しているが、原点（0,0,0）が画像の中央となる場合にのみ使用可能できる。<br>
ブレグマなど別の点を原点と指定した場合には修正が必要です。<br>
<br>
　(参照)[Resizing, reshaping and resampling nifti files](https://www.kaggle.com/mechaman/resizing-reshaping-and-resampling-nifti-files)

In [ ]:
def createCroppedImg(inputDir, inputFile, target_shape_for_affine, target_shape_for_dim, resolution, interpolation, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #target_shape_for_affine : Cropする大きさ（numpy.array形式、テンプレート画像のqoffset_x,y,zを解像度サイズで除して符号を反転したもの）
    #target_shape_for_dim : Cropする位置（numpy.array形式、テンプレート画像のヘッダーのdim情報＝画像のshapeのこと）
    #resolution : ボクセルサイズ（ここでダウン・アップサンプリングも可能）
    #interpolation: 'continuous' (default), 'linear', or 'nearest'
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')

    # Crop用の4x4変換行列を作製する
    new_affine = np.zeros((4,4))
    new_affine[:3,:3] = np.diag(resolution)
    new_affine[:3,3] = target_shape_for_affine*resolution/2.*-1
    new_affine[3,3] = 1.
    
    # Cropping (resampling)
    Croppednii = resample_img(niimg, 
                              target_affine=new_affine, 
                              target_shape=target_shape_for_dim, 
                              interpolation=interpolation)
    print('Completed: ' + '/' + inputFile + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(Croppednii, os.path.join(outputPathName))


def createCroppedImg_for_MultiFrame(inputDir, inputFile, target_shape_for_affine, target_shape_for_dim, resolution, interpolation, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #target_shape_for_affine : Cropする大きさ（numpy.array形式、テンプレート画像のqoffset_x,y,zを解像度サイズで除して符号を反転したもの）
    #target_shape_for_dim : Cropする位置（numpy.array形式、テンプレート画像のヘッダーのdim情報＝画像のshapeのこと）
    #resolution : ボクセルサイズ（ここでダウン・アップサンプリングも可能）
    #interpolation: 'continuous' (default), 'linear', or 'nearest'
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')

    # Crop用の4x4変換行列を作製する
    new_affine = np.zeros((4,4))
    new_affine[:3,:3] = np.diag(resolution)
    new_affine[:3,3] = target_shape_for_affine*resolution/2.*-1
    new_affine[3,3] = 1.
    
    # Cropping (resampling)
    first_single_img = index_img(niimg, 0)
    Croppednii = resample_img(first_single_img, 
                              target_affine=new_affine, 
                              target_shape=target_shape_for_dim, 
                              interpolation=interpolation)
    '''
    一旦、ダブルフレームにしておく（これはダミーフレームとして使う）
    resample_img()で作ったものは3Dなので、後の処理のことを考えてここでは4Dデータにしておく
    index_img(Croppednii, slice(0, 1))とすることで1フレームの4Dデータとすることができる
    index_img(Croppednii, 0)だと3Dデータになってしまうので注意
    '''
    Croppednii = concat_imgs([Croppednii, Croppednii])
    Croppednii = index_img(Croppednii, slice(0, 1))
    
    for next_single_img in iter_img(niimg):
        next_Croppednii = resample_img(next_single_img, 
                                  target_affine=new_affine, 
                                  target_shape=target_shape_for_dim, 
                                  interpolation=interpolation)
        next_Croppednii = concat_imgs([next_Croppednii, next_Croppednii])
        next_Croppednii = index_img(next_Croppednii, slice(0, 1))
        Croppednii = concat_imgs([Croppednii, next_Croppednii])
    
    # 頭の余計なフレームをそぎ落とす
    Croppednii = index_img(Croppednii, slice(1, Croppednii.shape[3]))
    print('Completed: ' + '/' + inputFile + '.nii,' + '---> saving now')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(Croppednii, os.path.join(outputPathName))

共通パラメータ（今回はSAMITに含まれるラットのVOIテンプレート画像に合わせる）を設定します。<br>
以下のコードをそのまま実行します。


In [ ]:
resolution_size = 0.2
target_shape_for_affine = np.array((94, 168, 140))
target_shape_for_dim = np.array((96, 120, 96))
new_resolution = [resolution_size, ]*3
interpolation = 'linear'

#### ＜テンプレート画像のヘッダー情報を見るための関数（補足）＞
　共通パラメータ決定のためのテンプレート画像のヘッダーを見るには以下の関数を使う。<br>
　テンプレート画像のヘッダー情報をもとに上記共通パラメータを決定する。

In [ ]:
def showHeaderNIfTI(inputDir, inputFile):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）

    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    
    print(niimg.header)

（使用例）
```
showHeaderNIfTI('./', 'SAMIT')
showHeaderNIfTI('./', 'Schwarz_T2w')
```

Cropping処理の使用例は以下になります。<br>
Dynamic画像とStatic画像で使用する関数が異なるので注意してください。

In [ ]:
in_outDir = './dynamic'
prefix = 'c_'
suffix = ''

createCroppedImg_for_MultiFrame(in_outDir, 'r_test_data-SUV', \
                                target_shape_for_affine, target_shape_for_dim, \
                                new_resolution, interpolation, in_outDir, prefix, suffix)

In [ ]:
in_outDir = './static'
prefix = 'c_'
suffix = ''

createCroppedImg(in_outDir, 'r_test_data-SUV', \
                 target_shape_for_affine, target_shape_for_dim, new_resolution, \
                 interpolation, in_outDir, prefix, suffix)

# 07. Smoothing処理
---
必要に応じてSmoothing処理を行います。<br>


In [ ]:
def createSmoothedImg(inputDir, inputFile, fwhm, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #fwhm : 単位はmm
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    
    # Smoothing
    Smoothednii = smooth_image(niimg, fwhm)
    print('Completed: ' + '/' + inputFile + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(Smoothednii, os.path.join(outputPathName))

以下、実行例

In [ ]:
# 共通パラメータ
fwhm = 0.5   # fwhm : 半値全幅(Full Width at Half Maximum)

in_outDir = './in_vivo_CT'
prefix = 's_'
suffix = ''

createSmoothedImg(in_outDir, 'c_r_test_CT_data', fwhm, in_outDir, prefix, suffix)

# 08. VOI解析
---
　SPMや3D Slicerを使ってVOI解析を行っても良い。<br>
<br>
　以下はSAMITのVOI解析出力結果と同じような形式になるように作成したVOI解析コードです（最低限の機能のみ。今後バージョンアップ予定）。<br>
<br>
　このコードでは全脳VOIは出力されません。<br>

In [ ]:
# 単一ファイル解析用
def exportVOIvalue(inputDir, FileName, label_img, label_txt, outputDir, prefix, suffix, outputFlag):
    # inputDir: 
    # FileName: 拡張子は含めない
    # outputDir: 出力ディレクトリパス
    # label_img: VOI template image (.nii)
    # label_txt: tab区切りのテキストファイル（SAMITのラベルファイルを参照）
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語
    # outputFlag: 0 -> ファイル出力, 1 -> オブジェクトをreturn
    
    # load NIfTI image
    tmpPathName = inputDir + '/' + FileName + '.nii'
    niimg = nib.load(tmpPathName)

    # img_to_signals_labels()が同じshapeかつ同じaffineでないと処理してくれないので修正する
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()

    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # load a label NIfTI image
    tmpPathName = inputDir + '/' + label_img + '.nii'
    labelimg = nib.load(tmpPathName)

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(ImgArryData, affine=labelimg.header.get_best_affine(), header=new_header)
        
    if len(niimg.shape)==3:
        # シングルフレームの場合
        # img_to_signals_labels()が4Dデータしか扱えないので一時的にダミー4Dにする
        niimg = concat_imgs([ConvertImg, ConvertImg])
        Flag_M_Frames = 0
    else:
        # マルチフレームの場合
        # なぜかこの処理をしないとaffineが完全一致しているにも関わらず、一致していないと言われてしまうので仕方なく実行
        niimg = concat_imgs([ConvertImg])
        Flag_M_Frames = 1

    
    # import VOI label
    tmpPathName = inputDir + '/' + label_txt + '.txt'
    label_pd = pd.read_csv(tmpPathName, sep='\t',header=None, skiprows=5)
    label_pd.columns = ['index', 'name']

    # result VOI
    # img_to_signals_labels()で得られるsignalsはnumpy.ndarray形式なのでflatten()を使って平坦化する
    resVOI = img_to_signals_labels(niimg, labelimg, mask_img=None, background_label=0, order='F', strategy='mean')
    resVOI = resVOI[0].flatten()
   
    # ファイル名pd.series
    frames = [0] # initialize
    for i in range(1, niimg.shape[3]+1):
        tmp_frames = [i]*len(label_pd)
        frames.extend(tmp_frames)
    frames = frames[1:]
    
    if Flag_M_Frames==0:
        resVOI = resVOI[:len(label_pd)] # ダブルフレームになっているのでシングルに修正
        #IDs = IDs[:len(label_pd)]
        frames = frames[:len(label_pd)]
        FileName_pd = pd.DataFrame({'File name': [FileName]*len(label_pd),
                                    #'File ID': IDs,
                                    'Frame No': frames})
    else:
        FileName_pd = pd.DataFrame({'File name': [FileName]*len(label_pd)*niimg.shape[3],
                                    #'File ID': IDs,
                                    'Frame No': frames})
        label_pd = pd.concat([label_pd]*niimg.shape[3], axis=0).reset_index(drop=True) # フレーム分だけ繰り返したらインデックスを修正
    
    # result VOI最終版
    resVOI_pd = pd.DataFrame({'mean': resVOI})

    # VOI解析結果の整形
    outputResVOI = pd.concat([FileName_pd, label_pd, resVOI_pd], axis=1)
    
    if outputFlag==0:
        # テキストとして出力
        outputPathName = outputDir + '/' + prefix + FileName + suffix + '.csv'
        outputResVOI.to_csv(outputPathName, header=True, index=False, sep=',')
        
    elif outputFlag==1:
        # オブジェクトとして出力
        return(outputResVOI)


# 複数ファイル解析用
def exportVOIvalues(inputDir, inputFileArray, label_img, label_txt, outputDir, outputFile, prefix, suffix):
    # inputDir: 
    #inputFileArray : 入力ファイル名（4D NIfTI形式）をnumpy.Array形式で与える
    # outputDir: 出力ディレクトリパス
    # outputFile: 出力ファイル名
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語

    # 結果格納用の空のpandasデータフレームを作製
    VOItbl = pd.DataFrame({'File name': [],
                           'File ID': [],
                           'Frame No': [],
                           'index': [],
                           'name': [],
                           'mean': []})
    
    # ファイルごとにVOI解析し、結果をVOItblに追加していく
    for FileName in range(0, len(inputFileArray)):
        tmpTbl = exportVOIvalue(inputDir, inputFileArray[FileName], label_img, label_txt, '', '', '', 1)
        #IDs = [FileName]*len(label_pd)*niimg.shape[3]
        VOItbl = pd.concat([VOItbl, tmpTbl], axis=0)
        print('Processed: ' + inputFileArray[FileName] + '.nii')

    # テキストとして出力
    outputPathName = outputDir + '/' + prefix + outputFile + suffix + '.csv'
    VOItbl.to_csv(outputPathName, header=True, index=False, sep=',')

' label_img 'と' label_txt 'に指定するVOIテンプレート（.niiと.txt）ファイルは引数' inputDir 'の中にあらかじめ用意しておきます。<br>
<br>
それ以外のフォルダにある場合はエラーとなります。<br>
<br>
今回使用する label_txt は `/spm12/toolbox/samit/Schwarz_rat/VOIs/SAMIT.txt` になります。<br>
これと同じフォーマットであればオリジナルの ラベルテキストファイルでも使用可能です。<br>
<br>
以下のコードは一例です。<br>
1ファイルのみVOI解析する場合と、2ファイル以上をまとめてVOI解析する場合で使用する関数が異なるので注意します。<br>
<br>
処理が完了すると指定した出力先フォルダにCSVファイルが出力されます。

In [ ]:
label_img = 'SAMIT'
label_txt = 'SAMIT'

prefix = 'VOI_'
suffix = ''

in_outDir = './static'

# 単一ファイルのみVOI解析する場合
exportVOIvalue(in_outDir, 'c_r_test_data-SUV', label_img, label_txt, in_outDir, prefix, suffix, 0)

# 複数ファイルをまとめてVOI解析する場合
exportVOIvalues(in_outDir, np.array(['c_r_test_data01-SUV',
                                     'c_r_test_data02-SUV',
                                     'c_r_test_data03-SUV']), 
                label_img, label_txt, in_outDir, 'test_data_all', prefix, suffix)